**This is the simplified Opensantions data in csv format**

In [1]:
#import packages
import requests
import pandas as pd
import numpy as np
import os
import re

# OFAC SDN

In [2]:
#I can access the data via pandas very easily, output already in tabular form
data = pd.read_csv('https://data.opensanctions.org/datasets/20240121/us_ofac_sdn/targets.simple.csv')
data

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,first_seen,last_seen,last_change
0,NK-22HtK7WrxZ2sU3rmhz6PuZ,Person,Michael Kuajien,Michael Kuajian;Michael Kuajien Duer Mayok,1979-01-01,ke;ss,Nairobi,NaN,SDN List - Executive Order 13818 (Global Magni...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2023-04-20T10:27:20,2024-01-21T22:10:01,2023-11-17T08:23:17
1,NK-22oMG6jqPQknWaMjzTn4hK,Organization,Limited Liability Company Garantiya,Garantiya OOO,NaN,ru,"bulvar Tverskoi, d. 15 str. 2, Moscow",5067746901426;7703610362,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2023-04-20T10:27:20,2024-01-21T22:10:01,2023-11-17T08:23:17
2,NK-23p2d4vMT5sJtQ845GyzJt,Organization,Scientific and Production Association of Measu...,AO NPO IT;Aktsionernoe Obschestvo Nauchno Proi...,NaN,ru,"2k4 Pionerskaya Str., 141074 Korolyov",1095018006555;5018139517,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2023-04-20T10:27:20,2024-01-21T22:10:01,2023-11-17T08:23:17
3,NK-23rgYEXa9AHtupZKgS8Tbc,Person,Bahram Ali SHAYESTEH,Bahrami Ali JADALI;Bahrami Ali SHAYESTEH,1958-06-13;1963-05-06;1963-08-06,de,"80331 Muenchen, Bayern",NaN,SDN List - Unknown,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2023-04-20T10:27:20,2024-01-21T22:10:01,2023-11-17T08:23:17
4,NK-24UMVkuxF3svsAtg55i3Zi,Organization,ALBAHR ALAAHMAR ENERGY FZE,البحر الأحمر للطاقة م م ح,NaN,ae,"P1-ELOB Office No. E-44G-29, Hamriyah Free Zon...",11670819;19438,SDN List - Executive Order 13846 (Iran),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2023-04-20T10:27:20,2024-01-21T22:10:01,2023-11-17T08:23:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13799,ofac-47225,Vessel,Peria,NaN,NaN,lr,NaN,5LIZ6;636022479;IMO 9322827,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2024-01-18T16:10:01,2024-01-21T22:10:01,2024-01-18T16:10:01
13800,ofac-47226,Vessel,Sara II,NaN,NaN,lr,NaN,5LJI4;636022546;IMO 9301615,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2024-01-18T16:10:01,2024-01-21T22:10:01,2024-01-18T16:10:01
13801,ofac-47227,Vessel,Sensus,NaN,NaN,lr,NaN,5LHJ7;636022146;IMO 9296585,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2024-01-18T16:10:01,2024-01-21T22:10:01,2024-01-18T16:10:01
13802,ofac-47228,Vessel,Uze,NaN,NaN,lr,NaN,5LHB3;636022072;IMO 9323338,SDN List - Executive Order 14024 (Russia),NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2024-01-18T16:10:01,2024-01-21T22:10:01,2024-01-18T16:10:01


In [9]:
#create urls for all relevant dates
date_list = pd.date_range(start='20210820',end='20220115',freq='D').strftime('%Y%m%d')
date_list

Index(['20210820', '20210821', '20210822', '20210823', '20210824', '20210825',
       '20210826', '20210827', '20210828', '20210829',
       ...
       '20220106', '20220107', '20220108', '20220109', '20220110', '20220111',
       '20220112', '20220113', '20220114', '20220115'],
      dtype='object', length=149)

In [10]:
#get all the urls for all the dates we need the data for
#make each day an individuallist so we can compare and match them
websites = []

for i in date_list:
    test = 'https://data.opensanctions.org/datasets/'+(i)+'/us_ofac_sdn/targets.simple.csv'
    websites.append(test)
print(websites)

['https://data.opensanctions.org/datasets/20210820/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210821/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210822/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210823/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210824/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210825/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210826/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210827/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210828/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210829/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210830/us_ofac_sdn/targets.simple.csv', 'https://data.opensanctions.org/datasets/20210831/us_ofac_sdn/targets.simple.csv', 'ht

In [11]:
#first check for missing days
for site in websites:
    try:
        response = requests.get(site)
        response.raise_for_status()  # Raises an HTTPError for bad response
    except requests.exceptions.RequestException as e:
        print(f"Error accessing {site}: {e}")
        continue

In [12]:
#now loop over the batches to get all the data and append the dataframes to one another
#this gives us a list of dataframes
#the loop must contain a date identifyer as a new column so each dataframe is marked with the retrieval date that is then needed to match the different days against each other
entities_list = []
date_pattern = r'/datasets/(\d{8})/'
for site in websites:
    data = pd.read_csv(site)
    match = re.search(date_pattern, site) #extract the date from the url
    if match:
        date = match.group(1)
        data['date_stamp'] = date #set an individual date stamp for each dataframe matching the url date
        #ata =  data.set_index('date_stamp') #set as an index
    entities_list.append(data)
res = pd.concat(entities_list)  # concatenate list of dataframes

In [13]:
res

,id,schema,name,aliases,birth_date,countries,addresses,identifiers,sanctions,phones,emails,dataset,last_seen,first_seen,date_stamp
0,ofac-sdn-10000,Person,Jhon Eidelber CANO CORREA,Alejandro CARDONA RIBILLAS;FLECHAS;Jhonny CANO,1963-12-13,co,"Calle 18 No. 8-16, Cartago, Valle, Colombia;Ca...",16217170;16455750,SDN List - Block - Program - Unknown - 2006-10-25,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-07-26 11:55:45,2021-08-20 20:39:54,20210820
1,ofac-sdn-10001,Organization,INVERSIONES MACARNIC PATINO Y CIA S.C.S.,NaN,NaN,co,"Calle 19 No. 9-50, Ofc. 505, Pereira, Risarald...",816005011-4,SDN List - Block - Program - Unknown - 2006-10-25,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-07-26 11:55:45,2021-08-20 20:39:54,20210820
2,ofac-sdn-10002,Organization,COMERCIALIZADORA DE CAFE DEL OCCIDENTE CODECAF...,CODECAFE,NaN,co,"Carrera 8 No. 23-09, Ofc. 903, Pereira, Risara...",816004106-0,SDN List - Block - Program - Unknown - 2006-10-25,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-07-26 11:55:45,2021-08-20 20:39:54,20210820
3,ofac-sdn-10003,Organization,GOMEZ MARIN LTDA.,NaN,NaN,co,"Calle 16 No. 1N-74, Cartago, Valle, Colombia;K...",800102465-1,SDN List - Block - Program - Unknown - 2006-10-25,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-07-26 11:55:45,2021-08-20 20:39:54,20210820
4,ofac-sdn-10004,Organization,GAVIOTAS LTDA.,HOTEL SIN PECADOS;HOTEL SIN PK2;MOTEL CAMPO AMOR,NaN,co,"Calle 4A No. 21-34 Circunvalar, Cartago, Valle...",800032092-7,SDN List - Block - Program - Unknown - 2006-10-25,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-07-26 11:55:45,2021-08-20 20:39:54,20210820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9007,Q714509,Person,Ryong Hae Choe,Ryong-hae Ch'oe,1950-01-15,NaN,NaN,NaN,Program - SDN List - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-15 08:55:59,20220115
9008,Q72993,Person,Pu Il Choe,Bu-il Choi;Pu-il Ch'oe,1944-03-06,NaN,NaN,NaN,Program - SDN List - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-15 08:55:59,20220115
9009,Q7680491,Person,耀宗 譚,Yaozong Tan;Yiu-chung Tam;耀宗 谭,1949-12-15,cn,Beijing,NaN,Program - SDN List - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-15 08:55:59,20220115
9010,Q9330536,Person,艾力更 ·依明巴海,Ailigeng Yimingbahai;Arken Imirbaki;艾力更 依明巴海,1953-09-01;1953-09-30,cn,Beijing,NaN,Program - SDN List - Block - Executive Order 1...,NaN,NaN,US OFAC Specially Designated Nationals (SDN) List,2021-09-30 11:39:21,2022-01-15 08:55:59,20220115


In [ ]:
#now match the different dates against each other and return an indicator per row for a new listing or delisting
# Sort the DataFrame based on date_stamp and id
res.sort_values(by=['id', 'date_stamp'], inplace=True)

# Identify new entries and deletions based on the index and columns you want to compare
new_entries = ~res.duplicated(subset=['id'], keep='first')
deletions = ~res.duplicated(subset=['id'], keep='last')

# Identify unchanged rows by checking for duplicates based on the id and date_stamp
unchanged_rows = ~new_entries & ~deletions

# Set the values of new_entry, deletion, and unchanged columns
res['new_entry'] = new_entries
res['deletion'] = deletions
res['unchanged'] = unchanged_rows

# Reset index for the final result
res.reset_index(drop=True, inplace=True)

In [ ]:
res

In [ ]:
#test thematching function
test = res.query('new_entry == True')
test

In [ ]:
res_deduplicated = res.copy()
res_deduplicated.sort_values(by=['id', 'date_stamp'], inplace=True)

# Identify new entries and deletions based on the index and columns you want to compare
new_entries = ~res_deduplicated.duplicated(subset=['id'], keep='first')
deletions = ~res_deduplicated.duplicated(subset=['id'], keep='last')

# Identify unchanged rows by checking for duplicates based on the id and date_stamp
unchanged_rows = ~new_entries & ~deletions

# Create new columns for first seen and last seen dates
res_deduplicated['listing_date'] = res_deduplicated['date_stamp'].where(new_entries)
res_deduplicated['delisting_date'] = res_deduplicated['date_stamp'].where(deletions)

# Add a 'month' column
res_deduplicated['month'] = pd.to_datetime(res_deduplicated['date_stamp']).dt.to_period('M')

# Deduplicate the entries (keep the first occurrence for each entity)
res_deduplicated = res_deduplicated.drop_duplicates(subset=['id'], keep='first')

# Reset index for the final result
res_deduplicated.reset_index(drop=True, inplace=True)

# Print or further analyze the deduplicated DataFrame 'res_deduplicated'
print(res_deduplicated)

In [ ]:
res_deduplicated

In [ ]:
#the countries are coded as iso2 codes -> transform to iso 3
#keep both countries? could search for "ru" over regex functions
print(res_deduplicated.countries.unique())
print(res_deduplicated.schema.unique())
print(res_deduplicated.listing_date.unique())

In [ ]:
list_regimes = list[res_deduplicated.sanctions.unique()]
print(list_regimes)
#but it doesnt show me all

In [ ]:
#get missingness
is_null = res_deduplicated.isnull().sum()
display(is_null)

In [ ]:
res_deduplicated.columns

In [ ]:
res_deduplicated = res_deduplicated.drop([ 'last_seen', 'first_seen','new_entry', 'deletion','unchanged'],axis = 1)

In [ ]:
res_deduplicated.to_csv("ofac_sdn_test.csv")